# Hands Signs Digits Classification Using Resnets

In [2]:
# import packages
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import h5py

%matplotlib inline
np.random.seed(1)
tf.random.set_seed(2)

In [3]:
# load and prepare data
train_dataset = h5py.File('data/train_signs.h5', 'r')
X_train_orig = np.array(train_dataset['train_set_x'][:])
Y_train_orig = np.array(train_dataset['train_set_y'][:])

test_dataset = h5py.File('data/test_signs.h5', 'r')
X_test_orig = np.array(test_dataset['test_set_x'][:])
Y_test_orig = np.array(test_dataset['test_set_y'][:])

classes = np.array(test_dataset['list_classes'][:])

Y_train_orig = Y_train_orig.reshape((1, Y_train_orig.shape[0]))
Y_test_orig = Y_test_orig.reshape((1, Y_test_orig.shape[0]))

X_train = X_train_orig / 255.
X_test = X_test_orig / 255.
print(X_train.shape)
print(X_test.shape)

Y_train = np.eye(6)[Y_train_orig.reshape(-1)]
Y_test = np.eye(6)[Y_test_orig.reshape(-1)]
print(Y_train.shape)
print(Y_test.shape)

(1080, 64, 64, 3)
(120, 64, 64, 3)
(1080, 6)
(120, 6)


In [4]:
# utility functions for residual blocks

def identity_block(X, f, filters):
    '''
    This block is used when the input activation has the same dimension as the output activation.
    The shortcut path skips over 3 layers.
    '''
    F1, F2, F3 = filters # number of filters for each layer
    X_shortcut = X # save the input value for the shortcut path
    
    # first layer
    X = layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    # second layer
    X = layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    # third layer
    X = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)

    # add main path with shortcut path and apply activation
    X = layers.Add()([X, X_shortcut])
    X = layers.Activation('relu')(X)

    return X

def convolution_block(X, f, filters, s=2):
    '''
    This block is used when the input activation doesn't have the same dimension as the output activation.
    '''
    F1, F2, F3 = filters # number of filters for each layer
    X_shortcut = X # save the input value for the shortcut path
    
    # first layer
    X = layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    # second layer
    X = layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    # third layer
    X = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)

    # shortcut path
    X_shortcut = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid')(X_shortcut)
    X_shortcut = layers.BatchNormalization(axis=3)(X_shortcut)

    # add main path with shortcut path and apply activation
    X = layers.Add()([X, X_shortcut])
    X = layers.Activation('relu')(X)

    return X

In [6]:
# build the ResNet50 model
def ResNet50(input_shape=(64, 64, 3), classes=6, training=False):
    '''
    The model uses multiple stages of residual connections that are implemented as blocks of convolutions and blocks of identity mappings.
    '''
    # pre stage
    input = layers.Input(shape=input_shape)
    X = layers.ZeroPadding2D((3, 3))(input)

    # stage 1  
    X = layers.Conv2D(filters=64, kernel_size=7, strides=2)(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)
    X = layers.MaxPooling2D(pool_size=3, strides=2)(X)

    # stage 2
    X = convolution_block(X, f=3, filters=[64, 64, 256], s=1)
    for i in range(2):
        X = identity_block(X, f=3, filters=[64, 64, 256])

    # stage 3
    X = convolution_block(X, f=3, filters=[128, 128, 512], s=2)
    for i in range(3):
        X = identity_block(X, f=3, filters=[128, 128, 512])

    # stage 4
    X = convolution_block(X, f=3, filters=[256, 256, 1024], s=2)
    for i in range(5):
        X = identity_block(X, f=3, filters=[256, 256, 1024])

    # stage 5
    X = convolution_block(X, f=3, filters=[512, 512, 2048], s=2)
    for i in range(2):
        X = identity_block(X, f=3, filters=[512, 512, 2048])

    # post stage
    X = layers.AveragePooling2D(pool_size=2)(X)
    X = layers.Flatten()(X)
    X = layers.Dense(classes, activation='softmax')(X)

    # create model
    model = keras.Model(inputs=input, outputs=X, name='ResNet50')

    return model

In [11]:
model = ResNet50(input_shape=(64, 64, 3), classes=6)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00015), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(X_train, Y_train, epochs=5, batch_size=32)

Epoch 1/5


2023-09-15 12:03:57.564317: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 53084160 exceeds 10% of free system memory.


34/34 [==============================] - 78s 2s/step - loss: 1.6980 - accuracy: 0.3472
Epoch 2/5
34/34 [==============================] - 63s 2s/step - loss: 1.1242 - accuracy: 0.5602
Epoch 3/5
34/34 [==============================] - 62s 2s/step - loss: 0.9108 - accuracy: 0.6593
Epoch 4/5
34/34 [==============================] - 62s 2s/step - loss: 0.6071 - accuracy: 0.7750
Epoch 5/5
34/34 [==============================] - 63s 2s/step - loss: 0.3795 - accuracy: 0.8611


Model has been trained on 5 epochs only because I'm currently running on CPU. It can be trained on more epochs for better results.